In [1]:
# use case on vito backend
from platform import python_version
print(python_version())



3.7.3


In [2]:
# connect to vito backend
VITO_DRIVER_URL = "https://openeo.vito.be/openeo/1.0/"
import openeo
print(openeo.__version__)
# initiate a session to the vito backend:
from openeo.metadata import CollectionMetadata, TemporalDimension, BandDimension, SpatialDimension
session = openeo.connect(VITO_DRIVER_URL).authenticate_basic(username="peter", password="peter123")

0.4.5a1


In [3]:
# upload s1a
backscatter = session.load_disk_collection('GTiff', str('/data/users/Public/driesj/openeo/eurac/S1A_IW_GRDH_*_015_VH_VV_eurac.tif'), options={'date_regex': '.*_(\d{4})(\d{2})(\d{2})T.*'})
import json
print(json.dumps(backscatter.graph,indent=2))
#we have to provide our own collection metadata, as it is loaded ad hoc by the backend
backscatter.metadata = CollectionMetadata(metadata={},dimensions=[TemporalDimension(name='t',extent=("20151106","20151212"))]) #, BandDimension(name='bands', )])

# how to provide metadata for the band dimension so that its accepted by filter bands?
#backscatter.metatata = CollectionMetadata(metadata={},dimensions=[BandDimension(name='bands',bands=['VH','VV'])])
#print(backscatter.metadata)
#backscatter.download("eurac_backscatter.tiff",format="GTiff")


{
  "loaddiskdata1": {
    "process_id": "load_disk_data",
    "arguments": {
      "format": "GTiff",
      "glob_pattern": "/data/users/Public/driesj/openeo/eurac/S1A_IW_GRDH_*_015_VH_VV_eurac.tif",
      "options": {
        "date_regex": ".*_(\\d{4})(\\d{2})(\\d{2})T.*"
      }
    },
    "result": true
  }
}


/opt/conda/envs/classification/lib/python3.7/site-packages/openeo-0.4.5a1-py3.7.egg/openeo/metadata.py:177: UserWarning: No cube:dimensions metadata


In [7]:
# module 1 ----
# FILTER BANDS DOESN'T WORK ON DISK CONNECTION SO FAR VIA BANDNAMES VV VH
# normalize vh
#vh = backscatter.filter_bands(["VH"]) # this doesn't work... band dim is missing
#tst = backscatter.filter_temporal("2015-11-06","2015-11-07") # this works... time dim has been added correctly
#vh_ref = vh.max_time() # more generically reduce, callback max
#vh_norm = vh.merge(vh_ref) # mergecubes(vh, vh_ref, overlap_resolver = "-") # 1:n case
#session.reduce
#tst = backscatter.reduce_dimension("max", dimension="t")

# normalize vv

# normalize both bands at once
backscatter_ref = backscatter.max_time()
backscatter_norm = backscatter.merge(backscatter_ref,overlap_resolver="subtract")
#print(json.dumps(backscatter_norm.graph,indent=2))
#backscatter_norm.download("norm.tif", format="GTiff") # gtiff cannot depict 3d data cube -> bands are there time is not there
#backscatter_norm.download("norm.ncdf", format="NETCDF") # produces error
#job = backscatter.send_job()
#res = job.start_and_wait().download_results()

# check if time dimension was there (if the values change compared to norm.tif there was one)
backscatter_norm_red = backscatter_norm.max_time()
#reducer=
#backscatter_norm_red = backscatter_norm.reduce_dimension(dimension="temporal", reducer=reducer)
#reducer = openeo.internal.graph_building.PGNode("subtract", arguments={"x": {"from_node": vh}, "y": {"from_node": vv}})
#datacube = mean.reduce_dimension(dimension="bands", reducer=reducer)

backscatter_norm_red.download("norm2.tif", format="GTiff")
print(json.dumps(backscatter_norm_red.graph,indent=2))


{
  "loaddiskdata1": {
    "process_id": "load_disk_data",
    "arguments": {
      "format": "GTiff",
      "glob_pattern": "/data/users/Public/driesj/openeo/eurac/S1A_IW_GRDH_*_015_VH_VV_eurac.tif",
      "options": {
        "date_regex": ".*_(\\d{4})(\\d{2})(\\d{2})T.*"
      }
    }
  },
  "reducedimension1": {
    "process_id": "reduce_dimension",
    "arguments": {
      "data": {
        "from_node": "loaddiskdata1"
      },
      "dimension": "t",
      "reducer": {
        "process_graph": {
          "max1": {
            "process_id": "max",
            "arguments": {
              "data": {
                "from_parameter": "data"
              }
            },
            "result": true
          }
        }
      }
    }
  },
  "mergecubes1": {
    "process_id": "merge_cubes",
    "arguments": {
      "cube1": {
        "from_node": "loaddiskdata1"
      },
      "cube2": {
        "from_node": "reducedimension1"
      },
      "overlap_resolver": {
        "process_grap

In [56]:
#print(session.list_processes())
print([process["id"] for process in session.list_processes()])


['array_apply', 'arccos', 'arcosh', 'power', 'last', 'subtract', 'not', 'cosh', 'artanh', 'is_valid', 'first', 'median', 'eq', 'absolute', 'arctan2', 'array_labels', 'divide', 'is_nan', 'all', 'round', 'min', 'any', 'gte', 'cos', 'between', 'count', 'xor', 'extrema', 'and', 'variance', 'or', 'sum', 'sin', 'sinh', 'product', 'exp', 'neq', 'sd', 'sort', 'normalized_difference', 'int', 'order', 'array_find', 'if', 'sqrt', 'add', 'e', 'arsinh', 'mean', 'array_filter', 'mod', 'multiply', 'lte', 'pi', 'ceil', 'tan', 'tanh', 'arctan', 'floor', 'array_element', 'clip', 'sgn', 'quantiles', 'arcsin', 'rearrange', 'array_contains', 'is_nodata', 'gt', 'ln', 'log', 'max', 'lt', 'load_collection', 'load_disk_data', 'apply_neighborhood', 'apply_dimension', 'save_result', 'apply', 'reduce_dimension', 'add_dimension', 'aggregate_temporal', 'aggregate_spatial', 'mask', 'mask_polygon', 'filter_temporal', 'filter_bbox', 'filter_bands', 'apply_kernel', 'ndvi', 'resample_spatial', 'resample_cube_spatial', '

In [ ]:
# upload lia 
lia = session.load_disk_collection('GTiff', str('/data/users/Public/driesj/openeo/eurac/S1A_IW_GRDH_20141205T171502_015_LIA_eurac.tif'), options={'date_regex': '.*_(\d{4})(\d{2})(\d{2})T.*'})
#we have to provide our own collection metadata, as it is loaded ad hoc by the backend
lia.metadata = CollectionMetadata(metadata={},dimensions=[TemporalDimension(name='t',extent=("20151106","20151212"))])
#lia.download("eurac_lia.tiff",format="GTiff")


In [ ]:
# upload modis snow
modis = session.load_disk_collection('GTiff', str('/data/users/Public/driesj/openeo/eurac/ERUAC_MODIS_SNOW_*_120000.tif'), options={'date_regex': '.*_(\d{4})(\d{2})(\d{2})_.*'})
#we have to provide our own collection metadata, as it is loaded ad hoc by the backend
modis.metadata = CollectionMetadata(metadata={},dimensions=[TemporalDimension(name='t',extent=("20151106","20151212"))])
#modis.download("eurac_snow.tiff",format="GTiff")